In [1]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
###############################################################################################
# Script_name: Data wrangling using Python                                                    #
# Objective: It reads the data from a specified file path, combines columns, converts         #
#            date formats, removes special characters, cleans the data by removing duplicate  #
#            records and replacing NULL values, and finally exports the processed data into   #
#            an SQL table in a MySQL database.                                                #
# Date created: 06/02/2024  Date modified: 06/02/2024                                         #
# Team name: The Rock Squad                                                                   #
################################################################################################
#  Data Wrangling process - Pseudocode                                                         #
# step 1:Import necessary libraries                                                            #
# step 2:Get current timestamp for logging                                                     #
# step 3:Configure logging                                                                     #
# step 4:Log the start of the data wrangling task                                              #
# step 5:Get the path to the desktop                                                           #
# step 6:Path to the configuration file                                                        #
# step 7:Read the config file to get SQL password and data path                                #
# step 8:Read the input data from the specified path                                           #
# step 9:Check if the DataFrame is successfully created                                        #
# step 10:Establish connection to MySQL database                                               #
# step 11:Export the DataFrame to a SQL table                                                  #
# step 12:Read the table from SQL server                                                       #
# step 13:Combine the columns with delimiters                                                  #
# step 14:Convert all date formats to a single format                                          #
# step 15:Remove the special character from the column                                         #
# step 16:Clean the data: Remove duplicate records and replace NULL values with 'Not Available #
# step 17:Export the DataFrame into an SQL table                                               #
################################################################################################

# Import necessary libraries
import re
import time
import logging
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
import pymysql
pymysql.install_as_MySQLdb()
import configparser
import os
 
# Configure logging
log_file = fr"C:\Users\Kaviyaa.Velraman\Documents\python\Data_wrangling\03_Log_File\Data_wrangling.log"

if os.path.exists(log_file):
    open(log_file, 'w').close()

# Get current timestamp for logging
ts = time.time()
formatted_time = time.strftime("%Y-%m-%d %H %M %S", time.localtime(ts))

logging.basicConfig(filename=log_file, level=logging.INFO, format='%(levelname)s - %(message)s - %(asctime)s', filemode='w')
 
# Log the start of the data wrangling task

logging.info(f"# STEP-1: The data wrangling task begins its execution at: {formatted_time}")
 
# Get the path to the desktop

desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
 
# Path to the configuration file

config_file_path = os.path.join(desktop_path, "Data_wrangling.ini")
 
# Read the config file to get SQL password and data path

logging.info("# STEP-2: Read the config file to get SQL password and data path")
config = configparser.ConfigParser()
config.read(config_file_path)
db_username = config['mysql']['user']
db_password = config['mysql']['password']
db_host = config['mysql']['host']
db_port = config['mysql']['port']
db_name = config['mysql']['database']
data_path = config['data']['file_path']
 
# Read the input data from the specified path
logging.info("# STEP-3: Read the input file to convert it to a dataframe.")
df = pd.read_excel(data_path) if re.search(r'\.xlsx$|\.xls$', data_path) else pd.read_csv(data_path) if re.search(r'\.csv$', data_path) else None

# Check if the DataFrame is successfully created
if df is None:
    logging.exception("# STEP-3.1: The file format is not csv, xls, or xlsx.")
else:
    logging.info('# STEP-3.1: Successfully read the input file and converted to dataframe.')
 
# Establish connection to MySQL database
logging.info("# STEP-4: Establishing connection to MySQL")
engine = create_engine(f'mysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')
logging.info("# STEP-4.1: Established connection to MySQL")
 
# Export the DataFrame to a SQL table
logging.info("# STEP-5: Exporting DataFrame to SQL Server")
table_name = 'employee_details'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)
logging.info("# STEP-5.1: Successfully exported the DataFrame to the SQL server.")
 
# Read the table from SQL server
logging.info("# STEP-6: Reading the table from SQL server")
df = pd.read_sql_table(table_name, con=engine)
logging.info("# STEP-6.1: Successfully read the table from SQL server.")
 
# Combine the columns with delimiters
logging.info("# STEP-7: Combining the columns")
df['Full Name'] = df[['FirstName', 'LastName']].fillna('').astype(str).agg(' '.join, axis=1)
df['Full Name'] = df['Full Name'].str.strip()
logging.info("# STEP-7.1: Successfully combined the columns")
 
# Convert all date formats to a single format
logging.info("# STEP-8: Convert all date formats to a single date format")
df['StartDate'] = df['StartDate'].fillna('').apply(lambda x: pd.to_datetime(x).strftime('%d-%m-%Y') if isinstance(x, str) and x and re.search(r'\d+', x) else x)
logging.info("# STEP-8.1: All date formats were successfully transformed into a single date format.")
 
# Remove the special character from the column
logging.info("# STEP-9: Remove the special characters")
df['UserID'] = df['ADEmail'].fillna('').apply(lambda x: re.sub(r'\W', '', x) if isinstance(x, str) else x)
logging.info("# STEP-9.1: Successfully removed the special characters")
 
# Clean the data: Remove duplicate records and replace NULL values with 'Not Available'
logging.info("# STEP-10: Cleaning the data: Replace NULL values with 'Not Available' and duplicate records")
df = df.replace('', np.nan).fillna('Not available')
df = df.drop_duplicates()
print(df)

logging.info("# STEP-10.1: Successfully removed duplicate records and replaced NULL values with 'Not Available'.")
 
# Export the DataFrame into an SQL table
logging.info("# STEP-11: Export the DataFrame into an SQL table")
df.to_sql('employee_transformed_table', con=engine, if_exists='replace', index=False)
logging.info("# STEP-11.1: Successfully export the DataFrame into an SQL table")
 
# Log the end of the data wrangling task
ts = time.time()
logging.info(f"# STEP-12: The data wrangling task ends its execution at: {time.strftime('%Y-%m-%d %H %M %S', time.localtime(ts))}")

      EmpID FirstName    LastName   StartDate       ExitDate  \
0      3427     Uriah     Bridges  20-09-2019  Not available   
1      3428     Paula       Small  11-02-2023  Not available   
2      3429    Edward        Buck  10-12-2018  Not available   
3      3430   Michael     Riordan  21-06-2021  Not available   
4      3431   Jasmine       Onque  29-06-2019  Not available   
...     ...       ...         ...         ...            ...   
2995   3422    Jakobe    Erickson  22-06-2022      07-Aug-22   
2996   3423    Adyson  Strickland  28-12-2020  Not available   
2997   3424   Annabel     Wilkins  09-12-2020      04-Aug-22   
2998   3425    Kendra       Braun  28-05-2019      23-Oct-21   
2999   3426     Chace        Kerr  27-04-2022  Not available   

                        Title          Supervisor  \
0     Production Technician I        Peter Oneill   
1     Production Technician I     Renee Mccormick   
2          Area Sales Manager      Crystal Walker   
3          Area Sal